In [1]:
from difflib import SequenceMatcher
import pandas as pd
import numpy as np
import json
import string
import re

from openpyxl import load_workbook

In [2]:
wb = load_workbook(filename='../sheets/ValueOps Work.xlsx')

ad_ban_raw = pd.read_excel(
    '../sheets/ValueOps Work.xlsx', sheet_name=wb.sheetnames[1])
ad_nat_raw = pd.read_excel(
    '../sheets/ValueOps Work.xlsx', sheet_name=wb.sheetnames[2])
white_space_raw = pd.read_excel(
    '../sheets/ValueOps Work.xlsx', sheet_name=wb.sheetnames[3])
target_raw = pd.read_excel(
    '../sheets/ValueOps Work.xlsx', sheet_name=wb.sheetnames[4])
leads_raw = pd.read_excel(
    '../sheets/ValueOps Work.xlsx', sheet_name=wb.sheetnames[5])

print(leads_raw.shape[0])
print(ad_ban_raw.shape[0])
print(ad_nat_raw.shape[0])
print(white_space_raw.shape[0])
print(target_raw.shape[0])

282
981
1006
53
114


In [3]:
leads = leads_raw[['Company Name']]
leads.head()

,Company Name
0,UPS
1,Wells Fargo Bank National Association
2,Workday
3,Accenture
4,Wells Fargo Bank National Association


In [4]:
ad_ban = ad_ban_raw.copy()
ad_ban.columns = ad_ban_raw.iloc[6]
ad_ban = ad_ban.iloc[7:,0:3]
ad_ban.reset_index(drop=True, inplace=True)
ad_ban['Clicks'] = ad_ban['Clicks'].apply(lambda x: x if type(x) == int else 0)
ad_ban.head(5)
ad_ban_map = {}
for index, row in ad_ban.iterrows():
  ad_ban_map[row['Business Name']] = {
    'Impressions': row['Impressions'],
    'Clicks': row['Clicks']
  }

In [5]:
ad_nat = ad_nat_raw.copy()
ad_nat = ad_nat.iloc[:,0:3]
ad_nat['Clicks'] = ad_nat['Clicks'].apply(lambda x: x if type(x) == int else 0)
ad_nat_map = {}
for index, row in ad_nat.iterrows():
  ad_nat_map[row['Business Name']] = {
    'Impressions': row['Impressions'],
    'Clicks': row['Clicks']
  }

In [6]:
white_space = white_space_raw[['Parent Account Name']]
white_space.head(5)

,Parent Account Name
0,Abbott Laboratories
1,ABN Amro Group N.V.
2,Accenture
3,Allianz SE
4,Amgen Inc.


In [7]:
target = target_raw[['Company Name']]
target.head(5)

,Company Name
0,American Airlines
1,Abbott Laboratories
2,Abbvie
3,Aetna
4,Accenture


In [8]:
def sim(a, b):
  return SequenceMatcher(None, a, b).ratio()


def clean_name(name):
  name = name.lower()
  name = name.translate(
      str.maketrans('', '', string.punctuation.replace("&", '')))
  name = name.strip()
  name = name.replace("  ", ' ')
  name = name.replace(' inc', '')
  name = name.replace(" llc", '')
  name = name.replace(" the", '')
  name = re.sub("\\[a-z]*", '', name)
  return name


def similar(a, b):
  a = clean_name(a)
  b = clean_name(b)
  cond0 = sim(a, b) > 0.90
  cond1 = a + " " in b or b in a
  cond2 = sim(a.split()[0], b.split()[0]) > 0.9
  cond3 = a.lower().startswith(b.split()[0])
  cond4 = b.lower().startswith(a.lower().split()[0])
  return cond0 or (cond0 and cond1) or cond2 or cond3 or cond4

In [9]:
leads = leads_raw['Company Name']
print(len(leads))
new_leads = [leads[0]]
for lead in leads:
  flag = False
  for l in new_leads:
    if similar(lead, l):
      flag = True
      break
  if not flag:
    new_leads.append(lead)
new_leads.append('RBC Capital Markets')
new_leads.append('State Farm Mutual Automobile Insurance Company')
new_leads.append('Tennessee Department of Health')
new_leads.append('Tennessee Department of Transportation')
print(len(new_leads))

282
81


In [10]:
ad_ban = []
ad_ban_impress = []
ad_ban_clicks = []

ad_nat = []
ad_nat_impress = []
ad_nat_clicks = []

c_white = []
c_target = []
for lead in new_leads:
  flag = False
  for comp in list(ad_ban_map.keys()):
    if similar(lead, comp):
      if not flag:
        ad_ban.append(True)
        ad_ban_impress.append(int(ad_ban_map[comp]['Impressions']))
        ad_ban_clicks.append(int(ad_ban_map[comp]['Clicks']))
        flag = True
      else:
        ad_ban_impress[-1] += int(ad_ban_map[comp]['Impressions'])
        ad_ban_clicks[-1] += int(ad_ban_map[comp]['Clicks'])
  if not flag:
    ad_ban.append(False)
    ad_ban_impress.append(0)
    ad_ban_clicks.append(0)
  flag = False
  for comp in list(ad_nat_map.keys()):
    if similar(lead, comp):
      if not flag:
        ad_nat.append(True)
        ad_nat_impress.append(int(ad_nat_map[comp]['Impressions']))
        ad_nat_clicks.append(int(ad_nat_map[comp]['Clicks']))
        flag = True
      else:
        ad_nat_impress[-1] += int(ad_nat_map[comp]['Impressions'])
        ad_nat_clicks[-1] += int(ad_nat_map[comp]['Clicks'])
  if not flag:
    ad_nat.append(False)
    ad_nat_impress.append(0)
    ad_nat_clicks.append(0)
  flag = False
  for comp in list(target['Company Name']):
    if similar(lead, comp):
      c_target.append(True)
      flag = True
      break
  if not flag:
    c_target.append(False)  
  flag = False
  for comp in list(white_space['Parent Account Name']):
    if similar(lead, comp):
      c_white.append(True)
      flag = True
      break
  if not flag:
    c_white.append(False)  

print(len(new_leads), len(ad_ban), len(ad_ban_impress), len(ad_ban_clicks), len(ad_nat), len(ad_nat_impress), len(ad_nat_clicks), len(c_target), len(c_white))
    


81 81 81 81 81 81 81 81 81


In [11]:
df = pd.DataFrame({
  'All Leads': new_leads,
  'AdDaptive Banners': ad_ban,
  'AdDaptive Banners Impressions': ad_ban_impress,
  'AdDaptive Banners Clicks': ad_ban_clicks,
  'AdDaptive Native': ad_nat,
  'AdDaptive Native Impressions': ad_nat_impress,
  'AdDaptive Native Clicks': ad_nat_clicks
})
df.to_csv(path_or_buf='./AdDaptive_Results.csv', index=False)

In [12]:
df2 = pd.DataFrame({
  'All Leads': new_leads,
  'AdDaptive Banners': ad_ban,
  'AdDaptive Native': ad_nat,
  'White Space': c_white,
  'Target List': c_target
})
df2.to_csv(path_or_buf='./Leads_Matching.csv', index=False)

In [13]:
c_impress = []
c_clicks = []
for c in list(target['Company Name']):
  flag = False
  for comp in list(ad_ban_map.keys()):
    if similar(c, comp):
      c_impress.append(c)
      if ad_ban_map[comp]['Clicks'] != 0:
        c_clicks.append(c)
      flag = True
      break
  if flag:
    continue
  for comp in list(ad_nat_map.keys()):
    if similar(c, comp):
      c_impress.append(c)
      if ad_ban_map[comp]['Clicks'] != 0:
        c_clicks.append(c)
      break

print('Total # of Companies in Target List: {}\n# of Companies with Impressions: {}\n# of Companies with Clicks: {}'.format(len(list(target['Company Name'])), len(c_impress), len(c_clicks)))

Total # of Companies in Target List: 114
# of Companies with Impressions: 103
# of Companies with Clicks: 50
